# Sanitize TSU list

For all entries which never were in the ESGF, sort each either into *error lists* or *suggest alternatives*.

In [1]:
import json
import pandas as pd
wrong_drs=pd.read_csv("cmip6_list_data_ref_syntax_wrong-drs-by-pids.txt")

In [2]:
drs_keys=['activity_id', 'institution_id', 'source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label', 'version']
drs_keys_noversion=['activity_id', 'institution_id', 'source_id', 'experiment_id', 'member_id', 'table_id', 'variable_id', 'grid_label']

In [3]:
wrong_drs=pd.DataFrame([k.split('.') for k in wrong_drs["data_ref_syntax"]], columns=drs_keys)

### 1. Sort out

Variables which are not defined correctly.

In [4]:
sort_out=wrong_drs[(wrong_drs["variable_id"].str.contains("\?")) |
          (wrong_drs["variable_id"]=='')]
sort_out.agg('.'.join,axis=1).to_csv("cmip6_list_data_ref_syntax_sortout-varwrong.txt", index=False)

In [5]:
wrong_drs=wrong_drs[~wrong_drs.index.isin(sort_out.index)]

### 2. Duplicates

Multiple wrong versions for a dataset (duplicates)

In [7]:
duplicates=wrong_drs[wrong_drs[drs_keys_noversion].duplicated(keep=False)]
len(duplicates)
duplicates[drs_keys]

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version
155,ScenarioMIP,AWI,AWI-CM-1-1-MR,ssp126,r1i1p1f1,Amon,pr,gn,20190930
156,ScenarioMIP,AWI,AWI-CM-1-1-MR,ssp126,r1i1p1f1,Amon,pr,gn,20191002
157,ScenarioMIP,AWI,AWI-CM-1-1-MR,ssp126,r1i1p1f1,Amon,ua,gn,20190728
158,ScenarioMIP,AWI,AWI-CM-1-1-MR,ssp126,r1i1p1f1,Amon,ua,gn,20190729
159,ScenarioMIP,AWI,AWI-CM-1-1-MR,ssp126,r1i1p1f1,Amon,ua,gn,20190730
...,...,...,...,...,...,...,...,...,...
821,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp585,r13i1p1f1,SImon,siconc,gn,20190712
825,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp585,r4i1p1f1,SImon,siconc,gn,
826,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp585,r4i1p1f1,SImon,siconc,gn,20190712
827,ScenarioMIP,EC-Earth-Consortium,EC-Earth3,ssp585,r9i1p1f1,SImon,siconc,gn,


In [8]:
duplicates[drs_keys].agg('.'.join,axis=1).to_csv(
    "cmip6_list_data_ref_syntax_sortout-versionduplicated.txt", index=False)

The snake ~ is the negating sign i.e. we use it to sort out:

In [9]:
#wrong_drs=wrong_drs[~wrong_drs[drs_keys_noversion].duplicated()]

### 3. Candidates

#### Without DRS element version

Suggestions from the pool can be deduced with intake. Intake also uses a dataframe underneath:

In [10]:
wrong_drs

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version
0,ScenarioMIP,CAS,FGOALS-g3,ssp245,r1i1p1f1,day,tas,gn,20190818-20190819
1,CMIP,CAS,FGOALS-f3-L,historical,r1i1p1f1,Ofx,areacello,gn,
2,CMIP,CAS,FGOALS-f3-L,historical,r1i1p1f1,Ofx,volcello,gn,
3,CMIP,CAS,FGOALS-f3-L,historical,r1i1p1f1,Omon,thetao,gn,
4,CMIP,CAS,FGOALS-g3,historical,r1i1p1f1,Ofx,volcello,gn,
...,...,...,...,...,...,...,...,...,...
855,ScenarioMIP,NCAR,CESM2,ssp245,r2i1p1f1,Amon,tas,gn,
856,ScenarioMIP,NCAR,CESM2,ssp245,r3i1p1f1,Amon,tas,gn,
857,ScenarioMIP,NCAR,CESM2,ssp585,r1i1p1f1,Amon,tas,gn,
858,ScenarioMIP,NCAR,CESM2,ssp585,r2i1p1f1,Amon,tas,gn,


In [11]:
import intake
cmip6_col = intake.open_esm_datastore("https://gitlab.dkrz.de/data-infrastructure-services/intake-esm/-/raw/master/esm-collections/cloud-access/dkrz_cmip6_disk.json")
cmip6_col.df.head() 

/work/bm0021/conda-envs/pyhandleenv/lib/python3.7/site-packages/fastprogress/fastprogress.py:102: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")
/work/bm0021/conda-envs/pyhandleenv/lib/python3.7/site-packages/intake_esm/core.py:84: DtypeWarning: Columns (21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  self._df, self.catalog_file = _fetch_catalog(self.esmcol_data, esmcol_obj, csv_kwargs)


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,...,frequency,time_reduction,long_name,units,realm,level_type,time_min,time_max,format,uri
0,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,c2h6,gn,NaN,v20200511,...,mon,mean,C2H6 Volume Mixing Ratio,mol mol-1,aerosol,alevel,185001.0,201412,netcdf,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,c2h6,gn,NaN,v20200511,...,mon,mean,C2H6 Volume Mixing Ratio,mol mol-1,aerosol,alevel,185001.0,201412.nc.modified,netcdf,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,c3h6,gn,NaN,v20200511,...,mon,mean,C3H6 volume mixing ratio,mol mol-1,aerosol,alevel,185001.0,201412,netcdf,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,c3h8,gn,NaN,v20200511,...,mon,mean,C3H8 volume mixing ratio,mol mol-1,aerosol,alevel,185001.0,201412,netcdf,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,hist-piAer,r1i1p1f1,AERmon,cdnc,gn,NaN,v20200522,...,mon,mean,Cloud Liquid Droplet Number Concentration,m-3,aerosol,alevel,185001.0,201412,netcdf,/work/ik1017/CMIP6/data/CMIP6/AerChemMIP/BCC/B...


In [13]:
cmip6_cat=cmip6_col.df[drs_keys]
#df is the dataframe under the catalog
#each dataset consists of many files. by keeping only one we get the list of datasets:
cmip6_cat=cmip6_col.df[drs_keys].drop_duplicates(keep="first").reindex()

By defining common *indices* in both dataframes, we can subset one with the other:

In [15]:
wrong_drs_idx=wrong_drs.set_index(drs_keys_noversion).index
cmip6_cat_idx=cmip6_cat.set_index(drs_keys_noversion).index

In [16]:
cat_candidates_version=cmip6_cat[cmip6_cat_idx.isin(wrong_drs_idx)]

The workflow is:

1. Adding a column *candidate* to the wrong_drs dataframe
1. Saving a json dictionary consisting of *wrong_drs_entry:candidate_drs* entries

In [17]:
cat_candidates_version

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version
3878,AerChemMIP,BCC,BCC-ESM1,ssp370,r1i1p1f1,Amon,tas,gn,v20190624
127840,AerChemMIP,NIMS-KMA,UKESM1-0-LL,ssp370-lowNTCF,r3i1p1f2,AERmon,o3,gn,v20210511
260637,CMIP,AS-RCEC,TaiESM1,historical,r1i1p1f1,Amon,pr,gn,v20200623
260662,CMIP,AS-RCEC,TaiESM1,historical,r1i1p1f1,Amon,ua,gn,v20200623
262726,CMIP,AS-RCEC,TaiESM1,historical,r1i1p1f1,SImon,siconc,gn,v20200630
...,...,...,...,...,...,...,...,...,...
6121647,ScenarioMIP,UA,MCM-UA-1-0,ssp245,r1i1p1f2,Amon,ua,gn,v20190731
6121673,ScenarioMIP,UA,MCM-UA-1-0,ssp370,r1i1p1f2,Amon,pr,gn,v20190731
6121684,ScenarioMIP,UA,MCM-UA-1-0,ssp370,r1i1p1f2,Amon,ua,gn,v20190731
6121711,ScenarioMIP,UA,MCM-UA-1-0,ssp585,r1i1p1f2,Amon,pr,gn,v20190731


In [18]:
cat_candidates_version_idx=cat_candidates_version.set_index(drs_keys_noversion).index

In [19]:
wrong_drs_candidates_version=wrong_drs[wrong_drs_idx.isin(cat_candidates_version_idx)].reset_index(drop=True)

In [20]:
wrong_drs_candidates_version_idx=wrong_drs_candidates_version.set_index(drs_keys_noversion).index

`.loc[ROW_INDICES,COLUMN_INDICES]` assigns new values to the existing data frame

In [21]:
wrong_drs_candidates_version.loc[:,"data_ref_syntax"]=wrong_drs_candidates_version.agg('.'.join,axis=1)
cat_candidates_version.loc[:,"data_ref_syntax"]=cat_candidates_version[drs_keys].agg('.'.join,axis=1)

/work/bm0021/conda-envs/pyhandleenv/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [22]:
for row,idx in enumerate(wrong_drs_candidates_version_idx) :
    wrong_drs_candidates_version.loc[row,"candidate"]=','.join(cat_candidates_version.set_index(drs_keys_noversion).loc[idx,"data_ref_syntax"].values)

In [23]:
wrong_drs_candidates_version_dict={}
for i,row in wrong_drs_candidates_version[["data_ref_syntax","candidate"]].iterrows():
    wrong_drs_candidates_version_dict[row["data_ref_syntax"]]=row["candidate"]

In [24]:
with open("cmip6_list_data_ref_syntax_drs-candidates-by-version.json", 'w') as f:
    json.dump(wrong_drs_candidates_version_dict,f,indent=4, sort_keys=True)

### 4. The rest

In [29]:
wrong_drs=wrong_drs[~wrong_drs_idx.isin(wrong_drs_candidates_version_idx)]
wrong_drs.agg('.'.join,axis=1).to_csv("cmip6_list_data_ref_syntax_sortout-notfound.txt", index=False)